## Create Sampled Dataset of KITTI


In [38]:
import argparse
import os
import yaml
import numpy as np
import open3d as o3d
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from matplotlib import cm
from functools import partial
import concurrent.futures
import random

from sphere import Sphere
from dh_grid import DHGrid
from laserscan import SemLaserScan

import os
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
import numpy as np

tf.enable_eager_execution()

from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.protos import segmentation_metrics_pb2
from waymo_open_dataset.protos import segmentation_submission_pb2

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
dataroot = '/media/scratch/berlukas/waymo'
FILENAME = f'{dataroot}/individual_files_training_segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord'


In [40]:
def convert_range_image_to_point_cloud_labels(frame,
                                              range_images,
                                              segmentation_labels,
                                              ri_index=0):
  """Convert segmentation labels from range images to point clouds.

  Args:
    frame: open dataset frame
    range_images: A dict of {laser_name, [range_image_first_return,
       range_image_second_return]}.
    segmentation_labels: A dict of {laser_name, [range_image_first_return,
       range_image_second_return]}.
    ri_index: 0 for the first return, 1 for the second return.

  Returns:
    point_labels: {[N, 2]} list of 3d lidar points's segmentation labels. 0 for
      points that are not labeled.
  """
  calibrations = sorted(frame.context.laser_calibrations, key=lambda c: c.name)
  point_labels = []
  for c in calibrations:
    range_image = range_images[c.name][ri_index]
    range_image_tensor = tf.reshape(
        tf.convert_to_tensor(range_image.data), range_image.shape.dims)
    range_image_mask = range_image_tensor[..., 0] > 0

    if c.name in segmentation_labels:
      sl = segmentation_labels[c.name][ri_index]
      sl_tensor = tf.reshape(tf.convert_to_tensor(sl.data), sl.shape.dims)
      sl_points_tensor = tf.gather_nd(sl_tensor, tf.where(range_image_mask))
    else:
      num_valid_point = tf.math.reduce_sum(tf.cast(range_image_mask, tf.int32))
      sl_points_tensor = tf.zeros([num_valid_point, 2], dtype=tf.int32)
      
    point_labels.append(sl_points_tensor.numpy())
  return point_labels

In [63]:
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
all_sem_clouds = []
k = 0
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    
    if frame.lasers[0].ri_return1.segmentation_label_compressed:
        (range_images, camera_projections, segmentation_labels, range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frame)
        points, cp_points = frame_utils.convert_range_image_to_point_cloud(frame, range_images, camera_projections, range_image_top_pose, keep_polar_features=True)
        point_labels = convert_range_image_to_point_cloud_labels(frame, range_images, segmentation_labels)
        
        points_all = np.concatenate(points, axis=0) # 3d points in vehicle frame.
        points_all = points_all[:, [3, 4, 5, 1]]
        point_labels_all = np.concatenate(point_labels, axis=0) # point labels.
        point_labels_all = np.reshape(point_labels_all[:,1], (point_labels_all.shape[0], 1))
        
        point_xyzil = np.hstack((points_all, point_labels_all))
        all_sem_clouds.append(point_xyzil)
        
print(f'Finished Loading {len(all_sem_clouds)} clouds.')

Finished Loading 30 clouds.


In [47]:
np.save(f'{dataroot}/test.npy', point_xyzil)

In [ ]:
all_sem_clouds = []

def progresser(sample_idx, grid, auto_position=True, write_safe=False, blocking=True, progress=False):    
#     sample_sphere = Sphere(sample)
    sample_sphere = all_sem_clouds[sample_idx]
    features = sample_sphere.sampleUsingGrid(grid)
    return features

def get_pointcloud_at(scan, name, label):
    pointclouds = []            
    scan.open_scan(name)
    scan.open_label(label)
    scan.colorize()

    pc = np.column_stack((scan.points, scan.remissions, scan.sem_label))
#     mask = pc[:,4] > 0 # Filter based on labeled data.        
    pointclouds.append(pc)
    return pointclouds

def get_map_at(scan, names, labels, indices):
    pointclouds = []    
    for idx in indices:
        scan.open_scan(names[idx])
        scan.open_label(labels[idx])
        scan.colorize()
        
        pc = np.column_stack((scan.points, scan.remissions, scan.sem_label))
        mask = pc[:,4] > 0 # Filter based on labeled data.        
        pointclouds.append(pc[mask])
    return pointclouds

def retrieve_poses_at(all_poses, indices):
    poses = []
    for idx in indices:
        poses.append(all_poses[idx])
    return poses

def combine_pointclouds(pointclouds, poses):
    n_data = len(poses)
    
    pivot = n_data // 2  
    T_G_L_pivot = poses[pivot]
    T_L_pivot_G = np.linalg.inv(T_G_L_pivot)

    acc_points = pointclouds[pivot]
    for i in range(0, n_data):
        if i == pivot:
            continue

        T_G_L = poses[i]
        T_L_pivot_L = T_L_pivot_G @ T_G_L

        points = Utils.transform_pointcloud(pointclouds[i], T_L_pivot_L)
        acc_points = np.append(acc_points, points, axis=0)                    
    
    return acc_points

CFG = load_config_file(config_file)
color_dict = CFG["color_map"]
nclasses = len(color_dict)
scan = SemLaserScan(nclasses, color_dict, project=False)
bw = 100
assert CFG is not None
  
grid, _ = DHGrid.CreateGrid(bw)
for seq in sequences:
    print(f'Loading sequence {seq}.')    
    scan_names, label_names = load_sequence(dataroot, seq)
    calib = parse_calibration(dataroot, seq)
    poses = parse_poses(dataroot, seq, calib)    
    n_scans = len(scan_names)    
    print(f'This sequence has {len(poses)} data elements.')
        
    all_sem_clouds = []
#     n_scans = 1000
    for i in range(0, n_scans):                
        pointcloud = get_pointcloud_at(scan, scan_names[i], label_names[i])        
        # all_sem_clouds.append(Sphere(pointcloud[0]))
        all_sem_clouds.append(pointcloud[0])
        
    print(f"Loading complete. Computing features...")
    # parallel
    sem_idx = np.arange(0, len(all_sem_clouds))
    sample_func = partial(progresser, grid=grid)
    sem_features = process_map(sample_func, sem_idx, max_workers=16)            

    filename = f"{export_ds}/clouds-{seq}.npy"
    np.save(filename, sem_features)
    print(f"Wrote features to {filename}.")

In [ ]:
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def visualizeRawPointcloud(pcl, val):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl[:, 0:3])
    colors = mapIntensityToRGB(val)
#     colors = scan.sem_color_lut[pcl[:,4].astype(np.int)]
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.visualization.draw_geometries([pcd])    
    
def createGrid_old(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([n_grid * n_grid, 2])
    for i in range(n_grid):
        for j in range(n_grid):
            points[k, 0] = (np.pi*(2*i+1))/(4*bw)
            points[k, 1] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])
    return cart_grid

def create_sampling_sphere(bw):
    grid = createGrid_old(bw)
    xyz_grid = convertGridToEuclidean_old(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid.T

In [ ]:
pc = all_sem_clouds[0].point_cloud
visualizeRawPointcloud(pc, pc[:, 4])

In [ ]:
sem_features[0,:,:,:].shape

In [ ]:
filename = f"{export_ds}/clouds-08-2.npy"
sem_features = np.load(filename)


cur_sem_cloud = sem_features[0, :, :, :]
cur_sem_cloud = np.reshape(cur_sem_cloud, (3, -1)).T
print(f'{cur_sem_cloud.shape}')
pc = create_sampling_sphere(bw)
points_xyz = pc.T[:,0:3]
print(f"sampling pointcloud shape is {points_xyz.shape}")
print(f"feature shape is {cur_sem_cloud.shape}")
points_xyzl = np.column_stack((points_xyz, cur_sem_cloud[:,2]))
# points_xyzn = np.column_stack((points_xyz, cur_sem_cloud[:,2]))

visualizeRawPointcloud(points_xyzl, points_xyzl[:, 3])